Hello! This is my first notebook and upload of a Dataset - as such any constructive feedback you have, I would love to hear. 

In future I want the form stats to loop over seasons for teams (this is doable, but runtime might be a bit of an issue) as well as getting data as to which players are playing for each team, thus we get more of a sense of how individual players affect team performance explicitly in the dataframe.

In [ ]:
# Here are the packages I loaded in for testing this on a Neural Network

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os

#Here is how we put all the seasons from 2014-2019 from the Premier League into one dataframe
#- so that we can use it as training data
team_csvs = []
Team_1420_dfs = []
years = ['/2014', '/2015', '/2016', '/2017', '/2018', '/2019', '/2020']
for year in years[:-1]:
    year = year[1:]
    for entry in os.scandir('../input/football-soccor-teams-stats-20142021/Understat_Team_Data/EPL_teams' + '/{}'.format(year)):
        if entry.is_file():
            df = pd.read_csv('../input/football-soccor-teams-stats-20142021/Understat_Team_Data/EPL_teams' + '/{}'.format(year) + '/{}'.format(entry.name))
            for col in list(df.columns):
                if 'Unnamed' in col:
                    del df[col]       
            Team_1420_dfs.append(df)
big_team_df = pd.concat(Team_1420_dfs, ignore_index=True)
big_team_df.to_csv('All_Epl_dfs_1420.csv')

Next, I shift the dataset so that we try to measure the probability of winning from the PREVIOUS results of both teams, hence we split our dataset in two halves, opponent dataset and previous game dataset, to get a result. Before that, I put in a dictionary so we can rename columns to show data from the previous game.

In [ ]:
OG_cols = ['Home/Away','xG','xGA','non-penalty xG','non-penalty xG Against','Deep','Deep Allowed','scored','conceded',
 'Expected Points','Result','date','Wins','Draws','Loses','Points','Non-penalty xG difference','ppda attack','ppda defence',
 'ppda allowed att','ppda allowed def','Wins cum','Loses cum','Draws cum','Points cum','npxG difference cum','xG cum','xGA cum',
 'npxG cum','npxGA cum','scored cum', 'conceded cum', 'Expected Points cum','Opponent','Prob Win',
 'Prob Draw','Prob Lose','Wins in last 5','Loses in last 5','Draws in last 5','Points in last 5','npxG difference last 5',
           'xG last 5','xGA last 5','npxG last 5','npxGA last 5','scored last 5','conceded last 5','Expected Points last 5']
prev_game_cols = []
for col in OG_cols:
    prev_game_cols.append('Prev game ' + col)

Here - we make the training set

In [ ]:
big_team_df = pd.read_csv('All_Epl_dfs_1420.csv')
#Make a df with JUST opponent data as well as the probabilities of win/lose/draw
for col in list(big_team_df.columns):
    if ('Opp' not in col) and ('Prob ' not in col):
        del big_team_df[col]
        
#Make the next one, where we keep the Team stat side of the dataframe, moving it to be the previous game.
big_team_df_2 = pd.DataFrame(index = [0], columns=big_team_df.columns)
big_team_df_2 = big_team_df_2.fillna(0)
for dataframe in Team_1420_dfs:
    jeff = []
    #Make the first line 0s 
    df_ = pd.DataFrame(index = [0], columns=big_team_df.columns)
    df_ = df_.fillna(0)
    #Get the above with the other lines of the dataframe
    jeff.append(df_)
    jeff.append(dataframe.drop(index = 37))
    #put all of these together
    jeff_concat = pd.concat(jeff, ignore_index = True)
    big_team_df_2 = pd.concat([big_team_df_2, jeff_concat], ignore_index = True)
big_team_df_2 = big_team_df_2.drop(index = 0)

#Delete all of the opponent columns/ probability columns
for col in list(big_team_df_2.columns):
    if ('Opp' in col) or ('Prob ' in col):
        del big_team_df_2[col]

prev_game_dict = {OG_cols[i]: prev_game_cols[i] for i in range(len(OG_cols))}
del prev_game_dict['Home/Away']
B_df_2 = big_team_df_2.rename(columns = prev_game_dict, inplace = False)
B_df_3 = pd.concat([B_df_2, big_team_df], axis = 1)

B_df_3 = B_df_3.dropna()

B_df_3['% Win'] = 100*B_df_3['Prob Win']
B_df_3['% Draw']= 100*B_df_3['Prob Draw']
B_df_3['% Lose']= 100*B_df_3['Prob Lose']
#Making sure the dataframe's columns are suitable for the model 
#- specifically assigning numerical values to indicate whether a team was home or away
h_a_dict = { 'h': 1, 'a' : -1}
def h_a_numbs(loc):
    return h_a_dict[loc]
B_df_3['Home/Away'] = B_df_3['Home/Away'].apply(h_a_numbs)

Here we are going to use the 20/21 Arsenal Season to validate our data. You could use any dataset you want from the 20/21 season to test a Neural Network with.

In [ ]:
df = pd.read_csv('../input/football-soccor-teams-stats-20142021/Understat_Team_Data/EPL_teams/2020/Arsenal.csv')
for col in list(df.columns):
    if ('Opp' not in col) and ('Prob ' not in col):
        del df[col]
#Previous Game data
df_1 = pd.read_csv('../input/football-soccor-teams-stats-20142021/Understat_Team_Data/EPL_teams/2020/Arsenal.csv')
df_ = pd.DataFrame(index = [0], columns=df.columns)
df_ = df_.fillna(0)
df_2 = pd.concat([df_, df_1.drop(index = 37)], ignore_index=True)
for col in list(df_2.columns):
    if ('Opp' in col) or ('Prob ' in col) or ('Unnamed' in col):
        del df_2[col]
prev_game_dict = {OG_cols[i]: prev_game_cols[i] for i in range(len(OG_cols))}
del prev_game_dict['Home/Away']
df_2 = df_2.rename(columns = prev_game_dict, inplace = False)
#Combining and making probabilities into percentages
Arsenal_2021_df = pd.concat([df_2, df], axis = 1)
Arsenal_2021_df = Arsenal_2021_df.drop(index = 0)
Arsenal_2021_df['% Win'] = 100*Arsenal_2021_df['Prob Win']
Arsenal_2021_df['% Draw']= 100*Arsenal_2021_df['Prob Draw']
Arsenal_2021_df['% Lose']= 100*Arsenal_2021_df['Prob Lose']
Arsenal_2021_df['Home/Away'] = Arsenal_2021_df['Home/Away'].apply(h_a_numbs)

Get rid of the columns that are strings and define our train/test split. I've not adjusted all columns to numerical values yet - only the home/away column as seen above.

In [ ]:
#Predicting 2020/21 season results from 2014-2020 results
cols = []
for col in list(B_df_3.columns):
    if type(B_df_3[col][2]) != str:
        cols.append(col)
cols.remove(cols[43])
cols.remove(cols[43])
cols.remove(cols[43])
cols = cols[:-3]
#cols.remove(cols[-2])
#cols.remove(cols[-2])
#cols.remove(cols[-2])
pred = ['% Win', '% Draw', '% Lose']
train_y = B_df_3['% Win']
train_X = B_df_3[cols]
val_y = Arsenal_2021_df['% Win']
val_X = Arsenal_2021_df[cols]

Scaling the model by using the Standard Scaler function as well as Principal Component Analysis (PCA)

In [ ]:
#Applying Scaling to the model, preprocessing the data
sc = StandardScaler()
sc.fit(train_X)
scaled_train_X = sc.transform(train_X)
scaled_val_X = sc.transform(val_X)

In [ ]:
pca = PCA(n_components = 30)
pca.fit(scaled_train_X)
X_pca = pca.transform(scaled_train_X)
X_pca_val = pca.transform(scaled_val_X)

Finally - we get on to a Neural Network! Batch Normalization and Dropout are applied here to eliminate a previous problem of overfitting. Other numbers I optimised through listing through certain values and seeing how that affected the results. Our Loss function ends up being valued at around 22-25 each time on the evaulation stage on the test data.

In [ ]:
# Let's try making a Neural Network! (With PCA)
early_stopping = EarlyStopping(
    min_delta=0.001,
    patience=20, 
    restore_best_weights=True,
)

NN_model_pca = keras.Sequential([
    layers.Dense(8, activation='relu', input_shape=[30]),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(8, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(8, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1),
])

NN_model_pca.compile(
    optimizer='adam',
    loss='mae',
)

history = NN_model_pca.fit(
    X_pca, train_y,
    validation_data=(X_pca_val, val_y),
    batch_size=190,
    epochs=500,
    verbose = 1,
    shuffle = True
)

Finally - we graph the history!

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

Thank you very much for reading, I would love to know your thoughts for how to optimise/ improve things in the comments! I am currently a student doing this in his spare time so I am open to any advice and feedback.